In [1]:
from util.GurobiOptimizer import GurobiOptimizer
import numpy as np

In [ ]:
# GurobiOptimizer = GurobiOptimizer()
# solution = GurobiOptimizer.run_full_optimization()

Processing series: 100%|██████████| 153/153 [13:13<00:00,  5.19s/series]


In [ ]:
initial_demands, final_grid_demands, charging_demands, soc_t_v_s, optimal_objectives, optimization_times, mip_gaps, self_sufficiencies, pv_utilizations = solution

In [5]:
print(np.array(self_sufficiencies).mean(), np.array(pv_utilizations).mean())
print(np.array(self_sufficiencies).std(), np.array(pv_utilizations).std())

0.4569165745086894 0.9664461290009903
0.10280459572448873 0.030298334374897238


In [ ]:
# import matplotlib.pyplot as plt
# from pyparsing import col

# print(sum(charging_demand))
# print([round(float(i),2) for i in pv])
# print(len(pv))
# plt.figure(figsize=(12, 6))
# plt.step(range(len(initial_demand)), initial_demand , label='Initial Demand', where='post')
# plt.step(range(len(final_grid_demand)), final_grid_demand, label='Final Demand', color='black', where='post')
# plt.step(range(len(charging_demand)), [final_grid_demand[i] - charging_demand[i] for i in range(len(charging_demand))], label='Charging Demand', color='orange', where='post')
# plt.xlabel('Time Step')
# plt.ylabel('Demand (kW)')
# plt.title('Demand Comparison')
# plt.legend()
# plt.grid(True)
# plt.show()

NameError: name 'charging_demand' is not defined

PLOT

In [2]:
from util.GurobiOptimizer import GurobiOptimizer
import numpy as np

GurobiOptimizer = GurobiOptimizer()
solution = GurobiOptimizer.run_full_immediate_charging()

Processing series (immediate): 100%|██████████| 153/153 [00:10<00:00, 14.81series/s]


In [3]:
initial_demands, final_grid_demands, charging_demands, soc_t_v_s, total_costs, self_sufficiencies, pv_utilizations = solution

In [8]:
RESULTS_PATH = "./output/V2_FIXED_WL48_PV500"
RESULTS_PATH_IMMEDIATE = "./output_immediate_V2_FIXED_WL48_PV500"

pv_utilizations = np.load(RESULTS_PATH + '/pv_utilizations.npy')
self_sufficiencies = np.load(RESULTS_PATH + '/self_sufficiencies.npy')
charging_demands = np.load(RESULTS_PATH + '/charging_demands.npy')
soc_t_v = np.load(RESULTS_PATH + '/soc_t_v.npy')

pv_utilizations_immediate = np.load(RESULTS_PATH_IMMEDIATE + '/pv_utilizations.npy')
self_sufficiencies_immediate = np.load(RESULTS_PATH_IMMEDIATE + '/self_sufficiencies.npy')
charging_demands_immediate = np.load(RESULTS_PATH_IMMEDIATE + '/charging_demands.npy')


In [9]:
# charging_demands.sum(axis=1)
soc_t_v.shape

(153, 199, 48)

In [10]:
soc_t_v_s[0][:,50].round(2)

array([0.72, 0.72, 0.9 , 0.9 , 0.9 , 0.9 , 0.9 , 0.9 , 0.9 , 0.9 , 0.9 ,
       0.9 , 0.9 , 0.9 , 0.9 , 0.9 , 0.9 , 0.9 , 0.9 , 0.9 , 0.9 , 0.9 ,
       0.9 , 0.9 , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  ])

In [11]:
soc_t_v[0][50].round(2)

array([0.72, 0.2 , 0.2 , 0.2 , 0.2 , 0.2 , 0.2 , 0.2 , 0.2 , 0.2 , 0.2 ,
       0.2 , 0.2 , 0.2 , 0.2 , 0.2 , 0.2 , 0.2 , 0.9 , 0.9 , 0.9 , 0.9 ,
       0.9 , 0.9 , 0.9 , 0.9 , 0.9 , 0.9 , 0.9 , 0.9 , 0.9 , 0.9 , 0.9 ,
       0.9 , 0.9 , 0.9 , 0.9 , 0.9 , 0.9 , 0.9 , 0.9 , 0.9 , 0.9 , 0.9 ,
       0.9 , 0.9 , 0.9 , 0.9 ])